In [1]:
import os
import numpy as np
import pandas as pd  # Potentially for labels if stored in a CSV file
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Define paths and labels (assuming class folders within 'flowers' directory)
base_dir = '../input/flowers-recognition/'
train_folder = os.path.join(base_dir, 'flowers')
labels = os.listdir(train_folder)  # Assuming class names correspond to folder names

In [3]:
# Set image size and other training parameters
image_size = 224
batch_size = 32  # Experiment with different batch sizes
epochs = 20  # Experiment with different numbers of epochs

In [4]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255  # Normalize pixel values between 0 and 1
    validation_split=0.2  # Use 20% of data for validation
)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255)

#

In [5]:
# Data augmentation with normalization
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2  # Use 20% of data for validation
)

In [6]:
# Define training and validation data generators
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = validation_datagen.flow_from_directory(
    train_folder,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=42  # Set a seed for reproducibility during validation split
)

Found 4317 images belonging to 5 classes.
Found 0 images belonging to 5 classes.


In [7]:
# Define the model architecture
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

def build_model():
    model = Sequential()
    # Load the pre-trained ResNet50 model with frozen layers
    base_model = ResNet50(weights=resnet_weights_path, include_top=False, input_shape=(image_size, image_size, 3))
    for layer in base_model.layers:
        layer.trainable = False  # Freeze pre-trained layers

    # Add custom layers for flower classification
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1024, activation='relu'))  # Experiment with number of units and activation
    model.add(Dropout(0.5))  # Experiment with dropout rate
    model.add(Dense(len(labels), activation='softmax'))  # Final layer with output size matching number of classes
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
model = build_model()
model.summary()  # View model architecture

# Early stopping and model checkpointing
early_stopper = EarlyStopping(monitor='val_loss', patience=5)
checkpointer = ModelCheckpoint(filepath='../working/best.hdf5',
                               monitor='val_loss',
                               save_best_only=True)

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5125      
Total params: 25,691,013
Trainable params: 2,103,301
Non-trainable params: 23,587,712
_________________________________________________________________


In [9]:
# Train the model with callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # Ensure all data seen per epoch
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,  # Ensure all validation data used
    epochs=epochs,
    callbacks=[early_stopper, checkpointer]
)
# Load the best model based on validation loss
model.load_weights('../working/best.hdf5')

Epoch 1/20
134/134 [==============================] - 689s 5s/step - loss: 0.6530 - acc: 0.7885
Epoch 2/20
134/134 [==============================] - 682s 5s/step - loss: 0.3863 - acc: 0.8590
Epoch 3/20
134/134 [==============================] - 690s 5s/step - loss: 0.3455 - acc: 0.8777
Epoch 4/20
134/134 [==============================] - 697s 5s/step - loss: 0.3206 - acc: 0.8889
Epoch 5/20
134/134 [==============================] - 695s 5s/step - loss: 0.2673 - acc: 0.9048
Epoch 6/20
134/134 [==============================] - 692s 5s/step - loss: 0.2644 - acc: 0.8994
Epoch 7/20
134/134 [==============================] - 693s 5s/step - loss: 0.2313 - acc: 0.9122
Epoch 8/20
134/134 [==============================] - 697s 5s/step - loss: 0.2235 - acc: 0.9171
Epoch 9/20
134/134 [==============================] - 698s 5s/step - loss: 0.2172 - acc: 0.9202
Epoch 10/20
134/134 [==============================] - 696s 5s/step - loss: 0.1792 - acc: 0.9334
Epoch 11/20
134/134 [==================

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ../working/best.hdf5

In [ ]:
print(history.history.keys())